https://mp.weixin.qq.com/s/dfNbvDizbigquizLQgZNug

In [1]:
import collections
import math
import os
import random
import zipfile
import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\jian

In [2]:
# 读取语料数据
filename = '../dataset/vecTraindata.zip'
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

In [10]:
# 根据语料，构建字典
def build_dataset(words):
    count = [['UNK',-1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word,_ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    data = [dictionary[word] if word in dictionary else 0 for word in words]
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))
    return data,count,dictionary,reverse_dictionary

In [11]:
# 为skip-gram模型生成训练批次
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1)
    for i in range(batch_size // num_skips):#i取值0,1,2,3
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [12]:
# Step 1: read data from zipdata
words = read_data(filename)
vocabulary_size = len(set(words))

In [13]:
print('Data size', vocabulary_size)

Data size 199247


In [14]:
# Step 2: Build the dictionary and replace rare words with UNK token.
data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 0], ('的', 99229), ('在', 25925), ('是', 20172), ('年', 17007)]
Sample data [14880, 4491, 483, 70, 1, 1009, 1850, 317, 14, 76] ['欧几里得', '西元前', '三', '世纪', '的', '希腊', '数学家', '现在', '被', '认为']


In [16]:
data_index = 0
# Step 3: Function to generate a training batch for the skip-gram model.

# Step 4: Build and train a skip-gram model.
# hyperparameters
batch_size = 128
embedding_size = 300 # dimension of the embedding vector
skip_window = 2 # how many words to consider to left and right
num_skips = 4 # how many times to reuse an input to generate a label

# we choose random validation dataset to sample nearest neighbors
# here, we limit the validation samples to the words that have a low
# numeric ID, which are also the most frequently occurring words
valid_size = 16 # size of random set of words to evaluate similarity on
valid_window = 100 # only pick development samples from the first 'valid_window' words
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64 # number of negative examples to sample

In [17]:
# create computation graph
graph = tf.Graph()

with graph.as_default():
    # input data
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # operations and variables
    # look up embeddings for inputs
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # construct the variables for the NCE loss
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each time we evaluate the loss.
    ncs_loss_test=tf.nn.nce_loss(weights=nce_weights, biases=nce_biases,
                   labels=train_labels, inputs=embed, num_sampled=num_sampled, num_classes=vocabulary_size)    
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases,
                     labels=train_labels, inputs=embed, num_sampled=num_sampled, num_classes=vocabulary_size))
    
    # construct the SGD optimizer using a learning rate of 1.0
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # compute the cosine similarity between minibatch examples and all embeddings
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # add variable initializer
    init = tf.initialize_all_variables()
#5
num_steps = 1000

with tf.Session(graph=graph) as session:
    # we must initialize all variables before using them
    init.run()
    print('initialized.')
    
    # loop through all training steps and keep track of loss
    average_loss = 0
  
    for step in range(num_steps):
        # generate a minibatch of training data
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        
        # we perform a single update step by evaluating the optimizer operation (including it
        # in the list of returned values of session.run())
        _, loss_val,ncs_loss_ = session.run([optimizer, loss,ncs_loss_test], feed_dict=feed_dict)
        average_loss += loss_val
        final_embeddings = normalized_embeddings.eval()
        print(final_embeddings)        
        print("*"*20)
        # print average loss every 2,000 steps
        #if step % 2000 == 0:
            #if step > 0:
                #average_loss /= 2000
            ## the average loss is an estimate of the loss over the last 2000 batches.
            #print("Average loss at step ", step, ": ", average_loss)
            #average_loss = 0
        
        ## computing cosine similarity (expensive!)
        #if step % 10000 == 0:
            #sim = similarity.eval()
            #for i in range(valid_size):
                ## get a single validation sample
                #valid_word = reverse_dictionary[valid_examples[i]]
                ## number of nearest neighbors
                #top_k = 8
                ## computing nearest neighbors
                #nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                #log_str = "nearest to %s:" % valid_word
                #for k in range(top_k):
                    #close_word = reverse_dictionary.get(nearest[k],None)
                    ##close_word = reverse_dictionary[nearest[k]]
                    #log_str = "%s %s," % (log_str, close_word)
                #print(log_str)
        
    final_embeddings = normalized_embeddings.eval()
    print(final_embeddings)
    fp=open('vector.txt','w',encoding='utf8')
    for k,v in reverse_dictionary.items():
        t=tuple(final_embeddings[k])
        #t1=[str(i) for i in t]
        s=''
        for i in t:
            i=str(i)
            s+=i+" "
            
        fp.write(v+" "+s+"\n")

    fp.close()
## Step 6: Visualize the embeddings.


#def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
    #assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    #plt.figure(figsize=(18, 18))  # in inches
    #for i, label in enumerate(labels):
        #x, y = low_dim_embs[i, :]
        #plt.scatter(x, y)
        #plt.annotate(label,
                 #xy=(x, y),
                 #xytext=(5, 2),
                 #textcoords='offset points',
                 #ha='right',
                 #va='bottom')

    #plt.savefig(filename)

#try:
    #from sklearn.manifold import TSNE
    #import matplotlib.pyplot as plt

    #tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    #plot_only = 500
    #low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    #labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    #plot_with_labels(low_dim_embs, labels)

#except ImportError:
    #print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use `tf.global_variables_initializer` instead.
initialized.
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.03538153 -0.0733199  -0.00298186 ... -0.04865492 -0.07313201
  -0.02488419]
 [ 0.08172591  0.08238789 -0.06541541 ...  0.00286334  0.09235063
   0.0503516 ]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.03304922 -0.06897052 -0.00629667 ... -0.04952673 -0.07674253
  -0.02883159]
 [ 0.08671682  0.08250338 -0.07090427 ...  0.00013652  0.09280491
   0.04988085]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859

[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.03455329 -0.08075061 -0.00707492 ... -0.04716816 -0.08665799
  -0.02739439]
 [ 0.09053043  0.07721553 -0.07744905 ... -0.00463357  0.08210339
   0.04764552]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.03800434 -0.07639953 -0.00944415 ... -0.04825576 -0.08469474
  -0.03258312]
 [ 0.09053043  0.07721553 -0.07744905 ... -0.00463357  0.08210339
   0.04764552]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.005689

[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.05068293 -0.07306848 -0.01466205 ... -0.03778323 -0.07421464
  -0.03740019]
 [ 0.09153427  0.06339251 -0.08835991 ... -0.01490815  0.07059448
   0.04849534]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.05235884 -0.07242979 -0.01300515 ... -0.03903763 -0.07255518
  -0.03902596]
 [ 0.09138083  0.06371062 -0.08800364 ... -0.0155504   0.07109113
   0.04819894]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.005689

   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.05935457 -0.0625747  -0.01510587 ... -0.04682317 -0.0738328
  -0.04088292]
 [ 0.09326494  0.05962053 -0.08729642 ... -0.01646235  0.06358533
   0.04379909]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.06023845 -0.06385051 -0.01468043 ... -0.0467176  -0.07332273
  -0.04073889]
 [ 0.09326494  0.05962053 -0.08729642 ... -0.01646235  0.06358533
   0.04379909]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ...

   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.06659692 -0.0534866  -0.01793899 ... -0.05137241 -0.06854121
  -0.03916213]
 [ 0.09123063  0.05989541 -0.08732814 ... -0.01743069  0.06359667
   0.04367395]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [-0.06630202 -0.05514899 -0.02101694 ... -0.05137819 -0.06956469
  -0.04166147]
 [ 0.09123063  0.05989541 -0.08732814 ... -0.01743069  0.06359667
   0.04367395]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ..

...

[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.01315877 -0.02113209 -0.0734858  ... -0.02209036 -0.08588865
   0.0413775 ]
 [ 0.08900329  0.04267865 -0.08713079 ... -0.01407812  0.02040306
   0.06548178]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.01444337 -0.01981313 -0.07306811 ... -0.02362558 -0.08541302
   0.04144091]
 [ 0.08900329  0.04267865 -0.08713079 ... -0.01407812  0.02040306
   0.06548178]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.005689

   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.01241707 -0.02893056 -0.07652048 ... -0.02503477 -0.086684
   0.04018042]
 [ 0.09189693  0.04355277 -0.08769997 ... -0.01466878  0.02069824
   0.0650126 ]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.01241707 -0.02893056 -0.07652048 ... -0.02503477 -0.086684
   0.04018042]
 [ 0.09189693  0.04355277 -0.08769997 ... -0.01466878  0.02069824
   0.0650126 ]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0

[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.0041721  -0.02969222 -0.08272234 ... -0.03064397 -0.08769893
   0.04054496]
 [ 0.08426251  0.04504466 -0.09067939 ... -0.01769088  0.02347561
   0.06554202]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.00449986 -0.03090046 -0.08268304 ... -0.02988896 -0.08586059
   0.0409916 ]
 [ 0.08426251  0.04504466 -0.09067939 ... -0.01769088  0.02347561
   0.06554202]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.005689

[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.01049566 -0.032887   -0.09281997 ... -0.03095561 -0.08005749
   0.04437349]
 [ 0.08683715  0.04272839 -0.09362856 ... -0.02046368  0.01865998
   0.06882635]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.01172055 -0.02849758 -0.09037222 ... -0.03064956 -0.08098074
   0.04443172]
 [ 0.08683715  0.04272839 -0.09362856 ... -0.02046368  0.01865998
   0.06882635]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.005689

[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.0017216  -0.02833268 -0.09301875 ... -0.02146222 -0.10975576
   0.03086053]
 [ 0.08268638  0.04151386 -0.09373065 ... -0.02064599  0.01610789
   0.06460064]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.00568939]]
********************
[[-0.02652693 -0.00898384  0.07751817 ...  0.01873513  0.01552536
   0.01536693]
 [ 0.00140717 -0.02771052 -0.09331766 ... -0.02147823 -0.10931653
   0.03078021]
 [ 0.08268638  0.04151386 -0.09373065 ... -0.02064599  0.01610789
   0.06460064]
 ...
 [ 0.06779774  0.06185494 -0.08426716 ...  0.01158162  0.04535786
  -0.04410859]
 [ 0.04647297  0.07357961 -0.05625805 ... -0.03646094 -0.04935031
   0.05482143]
 [-0.00672964 -0.08916941  0.09387613 ... -0.04424572 -0.08921115
   0.005689